In [12]:
import sys
import pandas as pd
import pymongo
import json
import os
import pprint
import numpy as np
from pandas.io.json import json_normalize
from sqlalchemy import create_engine

In [37]:
file = 'globalterrorismdb_0718dist.csv'
terror_df = pd.read_csv(file, encoding = "ISO-8859-1")

terror_df.head()

C:\Users\mjone\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [38]:
heat_df = terror_df[["latitude","longitude","eventid"]]
heat_df.head()

,latitude,longitude,eventid
0,18.456792,-69.951164,197000000001
1,19.371887,-99.086624,197000000002
2,15.478598,120.599741,197001000001
3,37.997490,23.762728,197001000002
4,33.580412,130.396361,197001000003


In [39]:
heat_df.set_index('eventid',inplace=True)
heat_df.head()

,latitude,longitude
eventid,,
197000000001,18.456792,-69.951164
197000000002,19.371887,-99.086624
197001000001,15.478598,120.599741
197001000002,37.997490,23.762728
197001000003,33.580412,130.396361


In [40]:
zoom_df = terror_df[["region","country_txt","provstate"]]
zoom_df.set_index('country_txt',inplace=True)
zoom_df.head()

,region,provstate
country_txt,,
Dominican Republic,2,NaN
Mexico,1,Federal
Philippines,5,Tarlac
Greece,8,Attica
Japan,4,Fukouka


In [41]:
bar_df=terror_df.groupby(['natlty1_txt', 'gname', ]).size().unstack(level=1, fill_value=0)
bar_df.head()

gname,Afghan Guerrillas,Afghan Mujahideen,Afghan Rebels,Al-Aqsa Martyrs Brigade,Al-Fatah,Al-Qaida,Al-Qaida in the Islamic Maghreb (AQIM),Crypteia,Gholam Yahya Akbar,Guerrillas,...,Congolese National Movement- Lumumba (MNCL),Supporters Opposition Leader Etienne Tshisekedi,Black Mambas,United Party for National Development (UPND),Dissidents linked to Zanga,Liberation War Veterans Association,Movement for Democratic Change (MDC),National Youth Service of Zimbabwe,South African guerrillas,Supporters of Joshua Nkomo
natlty1_txt,,,,,,,,,,,,,,,,,,,,,
Afghanistan,6,7,1,3,1,17,2,1,1,6,...,0,0,0,0,0,0,0,0,0,0
Albania,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Algeria,0,0,0,0,0,0,172,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Angola,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
line_df=terror_df.groupby(['iyear', 'country_txt']).size().unstack(level=1, fill_value=0)
line_df.head()

country_txt,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Vietnam,Wallis and Futuna,West Bank and Gaza Strip,West Germany (FRG),Western Sahara,Yemen,Yugoslavia,Zaire,Zambia,Zimbabwe
iyear,,,,,,,,,,,,,,,,,,,,,
1970,0,0,0,0,0,0,21,0,1,0,...,0,0,0,20,0,0,0,0,0,0
1971,0,0,0,0,0,0,7,0,1,0,...,0,0,0,13,0,0,0,0,1,0
1972,0,0,1,0,0,0,20,0,8,7,...,0,0,1,24,0,0,0,3,0,0
1973,1,0,0,0,0,0,60,0,0,2,...,0,0,0,27,0,0,1,1,1,0
1974,0,0,0,1,0,0,71,0,1,1,...,0,0,0,28,0,0,1,0,0,0


In [56]:
line_df[rate_of_change]=terror_df.groupby(['iyear', 'country_txt']).count().unstack(level=1, fill_value=0)
line_df.head()

eventid                                                     \
country_txt Afghanistan Albania Algeria Andorra Angola Antigua and Barbuda   
iyear                                                                        
1970                  0       0       0       0      0                   0   
1971                  0       0       0       0      0                   0   
1972                  0       0       1       0      0                   0   
1973                  1       0       0       0      0                   0   
1974                  0       0       0       1      0                   0   

                                                  ...    related  \
country_txt Argentina Armenia Australia Austria   ...    Vietnam   
iyear                                             ...              
1970               21       0         1       0   ...          0   
1971                7       0         1       0   ...          0   
1972               20       0         8       7   ...          0   
1973               60       0         0       2   ...          0   
1974               71       0         1       1   ...          0   

                                                                           \
country_txt Wallis and Futuna West Bank and Gaza Strip West Germany (FRG)   
iyear                                                                       
1970                        0                        0                  1   
1971                        0                        0                  0   
1972                        0                        0                  0   
1973                        0                        0                  0   
1974                        0                        0                  0   

                                                                   
country_txt Western Sahara Yemen Yugoslavia Zaire Zambia Zimbabwe  
iyear                                                              
1970                     0     0          0     0      0        0  
1971                     0     0          0     0      0        0  
1972                     0     0          0     3      0        0  
1973                     0     0          0     0      0        0  
1974                     0     0          0     0      0        0  

[5 rows x 27265 columns]

In [43]:
linear_df=terror_df.groupby(["country_txt", "iyear"]).count()
linear_df.head()
linear_df.to_csv('linear_df.csv')

In [44]:
freq_df = terror_df['country_txt'].value_counts().reset_index()
freq_df.columns = ['country_txt', 'count']
print (freq_df)

                        country_txt  count
0                              Iraq  24636
1                          Pakistan  14368
2                       Afghanistan  12731
3                             India  11960
4                          Colombia   8306
5                       Philippines   6908
6                              Peru   6096
7                       El Salvador   5320
8                    United Kingdom   5235
9                            Turkey   4292
10                          Somalia   4142
11                          Nigeria   3907
12                         Thailand   3849
13                            Yemen   3347
14                            Spain   3249
15                        Sri Lanka   3022
16                    United States   2836
17                          Algeria   2743
18                           France   2693
19                            Egypt   2479
20                          Lebanon   2478
21                            Chile   2365
22         

In [49]:
count = terror_df.groupby(['country_txt']).count() 
print(count)

                          eventid  iyear  imonth   iday  approxdate  extended  \
country_txt                                                                     
Afghanistan                 12731  12731   12731  12731        1099     12731   
Albania                        80     80      80     80           0        80   
Algeria                      2743   2743    2743   2743          77      2743   
Andorra                         1      1       1      1           0         1   
Angola                        499    499     499    499           3       499   
Antigua and Barbuda             1      1       1      1           0         1   
Argentina                     815    815     815    815           1       815   
Armenia                        24     24      24     24           0        24   
Australia                     114    114     114    114           0       114   
Austria                       115    115     115    115           2       115   
Azerbaijan                  

In [50]:
terror_df

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
5,197001010002,1970,1,1,NaN,0,NaN,217,United States,1,...,"The Cairo Chief of Police, William Petersen, r...","""Police Chief Quits,"" Washington Post, January...","""Cairo Police Chief Quits; Decries Local 'Mili...","Christopher Hewitt, ""Political Violence and Te...",Hewitt Project,-9,-9,0,-9,NaN
6,197001020001,1970,1,2,NaN,0,NaN,218,Uruguay,3,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
7,197001020002,1970,1,2,NaN,0,NaN,217,United States,1,...,"Damages were estimated to be between $20,000-$...",Committee on Government Operations United Stat...,"Christopher Hewitt, ""Political Violence and Te...",NaN,Hewitt Project,-9,-9,0,-9,NaN
8,197001020003,1970,1,2,NaN,0,NaN,217,United States,1,...,The New Years Gang issue a communiqué to a loc...,"Tom Bates, ""Rads: The 1970 Bombing of the Army...","David Newman, Sandra Sutherland, and Jon Stewa...","The Wisconsin Cartographers' Guild, ""Wisconsin...",Hewitt Project,0,0,0,0,NaN
9,197001030001,1970,1,3,NaN,0,NaN,217,United States,1,...,"Karl Armstrong's girlfriend, Lynn Schultz, dro...",Committee on Government Operations United Stat...,"Tom Bates, ""Rads: The 1970 Bombing of the Army...","David Newman, Sandra Sutherland, and Jon Stewa...",Hewitt Project,0,0,0,0,NaN
